In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import json
from IPython import display
from sklearn.model_selection import train_test_split
%matplotlib inline

In [39]:
# load the data
eeg_raw = pd.read_csv(os.path.join("data","eeg-data.csv"))
eeg_raw['eeg_power'] = eeg_raw.eeg_power.map(json.loads)
stimulus_times = pd.read_csv(os.path.join("data","stimulus-times.csv"))
metadata = pd.read_csv(os.path.join("data", "subject-metadata.csv"))

# drop the rows with signal quality greater than 128, as that indicates the headset is not worn properly
idx = eeg_raw.signal_quality <= 128
eeg = eeg_raw[idx]

# drop the unlabeled ys as those are useless
idx = eeg.label != 'unlabeled'
eeg = eeg[idx]

# clean up the data
y = eeg.label
eeg = eeg.drop(["Unnamed: 0", "indra_time","browser_latency", "reading_time","createdAt","updatedAt", "attention_esense", "meditation_esense","raw_values","signal_quality","label"], axis=1)

# split the eeg readings into columns
eeg[['eeg1','eeg2','eeg3','eeg4','eeg5','eeg6','eeg7', 'eeg8']] = pd.DataFrame(eeg.eeg_power.values.tolist(), index= eeg.index)

# drop the eeg power
eeg = eeg.drop(['eeg_power'], axis=1)

In [40]:
# transform the metadata
metadata = metadata.drop(["Session","Saw icons?","Chosen color"],axis=1)
metadata.columns = ["id","seen_video","gender","wear_contacts"]

In [41]:
# add the subject metadata
eeg = eeg.merge(metadata, on="id")

# drop the id
eeg = eeg.drop(["id"], axis=1)

In [42]:
# transform the data
eeg.seen_video = (eeg.seen_video == 'y') * 1.0
eeg.wear_contacts = (eeg.wear_contacts == 'y') * 1.0
eeg = pd.get_dummies(eeg, columns=["gender"])

In [43]:
# map the labels so that we use the general task rather than the instance of that task
np.unique(y)
map_dict = {
    'blinkInstruction':'instruction',
    'blink1':'blink',
    'blink2':'blink',
    'blink3':'blink',
    'blink4':'blink',
    'blink5':'blink',
    'colorInstruction1':'instruction',
    'colorInstruction2':'instruction',
    'colorRound1-1':'colorRound',
    'colorRound1-2':'colorRound',
    'colorRound1-3':'colorRound',
    'colorRound1-4':'colorRound',
    'colorRound1-5':'colorRound',
    'colorRound1-6':'colorRound',
    'colorRound2-1':'colorRound',
    'colorRound2-2':'colorRound',
    'colorRound2-3':'colorRound',
    'colorRound2-4':'colorRound',
    'colorRound2-5':'colorRound',
    'colorRound2-6':'colorRound',
    'colorRound3-1':'colorRound',
    'colorRound3-2':'colorRound',
    'colorRound3-3':'colorRound',
    'colorRound3-4':'colorRound',
    'colorRound3-5':'colorRound',
    'colorRound3-6':'colorRound',
    'colorRound4-1':'colorRound',
    'colorRound4-2':'colorRound',
    'colorRound4-3':'colorRound',
    'colorRound4-4':'colorRound',
    'colorRound4-5':'colorRound',
    'colorRound4-6':'colorRound',
    'colorRound5-1':'colorRound',
    'colorRound5-2':'colorRound',
    'colorRound5-3':'colorRound',
    'colorRound5-4':'colorRound',
    'colorRound5-5':'colorRound',
    'colorRound5-6':'colorRound',
    'colorRound6-1':'colorRound',
    'colorRound6-2':'colorRound',
    'colorRound6-3':'colorRound',
    'colorRound6-4':'colorRound',
    'colorRound6-5':'colorRound',
    'colorRound6-6':'colorRound',
    'math1':'math',
    'math2':'math',
    'math3':'math',
    'math4':'math',
    'math5':'math',
    'math6':'math',
    'math7':'math',
    'math8':'math',
    'math9':'math',
    'math10':'math',
    'math11':'math',
    'math12':'math',
    'readyRound1':'readyRound',
    'readyRound2':'readyRound',
    'readyRound3':'readyRound',
    'readyRound4':'readyRound',
    'readyRound5':'readyRound',
    'thinkOfItems-ver1':'thinkOfItems',
    'thinkOfItems-ver2':'thinkOfItems',
    'thinkOfItemsInstruction-ver1':'instruction',
    'thinkOfItemsInstruction-ver2':'instruction',
    'video-ver1':'video',
    'video-ver2':'video',
    'relaxInstruction':'instruction',
    'videoInstruction':'instruction',
    'mathInstruction':'instruction',
    'musicInstruction':'instruction',
}
new_y = np.copy(y)
for k, v in map_dict.items(): new_y[y==k] = v

In [44]:
# remove instructions
instr_idx = new_y != 'instruction'

new_y = new_y[instr_idx]
eeg = eeg[instr_idx]

In [45]:
# split the data into train and validation
X_tr, X_te, y_tr, y_te = train_test_split(eeg, new_y, test_size=0.25, random_state=1)

# split test into test and validation
X_te, X_cv, y_te, y_cv = train_test_split(X_te, y_te, test_size=0.5, random_state=1)

print("X_tr", X_tr.shape)
print("X_cv", X_cv.shape)
print("X_te", X_te.shape)
print("y_tr", y_tr.shape)
print("y_cv", y_cv.shape)
print("y_te", y_te.shape)

X_tr (5839, 12)
X_cv (974, 12)
X_te (973, 12)
y_tr (5839,)
y_cv (974,)
y_te (973,)


## Try to classify with an SVM as a baseline

In [31]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', None),
    ('svm', LinearSVC())
])

grid = {
    'svm__C': [0.125, 0.25, 0.375, 0.5],
}

grid_cv = GridSearchCV(pipe, grid, cv=3)

grid_cv.fit(X_tr, y_tr)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', None), ('svm', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svm__C': [0.125, 0.25, 0.375, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [32]:
# Collect results and sort them
df = pd.DataFrame.from_items([
    ('C', grid_cv.cv_results_['param_svm__C']),
    ('mean_accuracy', grid_cv.cv_results_['mean_test_score']),
    ('std', grid_cv.cv_results_['std_test_score'])
])

df.sort_values(by='mean_accuracy', ascending=False).head(10)

,C,mean_accuracy,std
0,0.125,0.241993,0.070066
3,0.5,0.218017,0.050206
2,0.375,0.198835,0.077987
1,0.25,0.164754,0.028465


In [ ]:
## Not very good results

## Try with a Neural Network

In [46]:
# transform labels into integers
names = np.unique(new_y)
i = 0
for name in names:
    new_y[new_y == name] = i
    i += 1

In [47]:
# split the data into train and validation
X_tr, X_te, y_tr, y_te = train_test_split(eeg.values, new_y, test_size=0.25, random_state=1)

# split test into test and validation
X_te, X_cv, y_te, y_cv = train_test_split(X_te, y_te, test_size=0.5, random_state=1)

print("X_tr", X_tr.shape)
print("X_cv", X_cv.shape)
print("X_te", X_te.shape)
print("y_tr", y_tr.shape)
print("y_cv", y_cv.shape)
print("y_te", y_te.shape)

X_tr (5839, 12)
X_cv (974, 12)
X_te (973, 12)
y_tr (5839,)
y_cv (974,)
y_te (973,)


In [48]:
# Batch generator
def get_batches(X, y, batch_size):
    # Shuffle X,y
    shuffled_idx = np.arange(len(y)) # 1,2,...,n
    np.random.shuffle(shuffled_idx)

    # Enumerate indexes by steps of batch_size
    # i: 0, b, 2b, 3b, 4b, .. where b is the batch size
    for i in range(0, len(y), batch_size):
        # Batch indexes
        batch_idx = shuffled_idx[i:i+batch_size]
        yield X[batch_idx], y[batch_idx]

In [52]:
graph = tf.Graph()
model_name = 'model_1'

with graph.as_default():
    # Create placeholders
    X = tf.placeholder(dtype=tf.float32, shape=[None, 12])
    y = tf.placeholder(dtype=tf.int32, shape=[None])
    training = tf.placeholder(dtype=tf.bool)
    
    # create variables
    global_step = tf.Variable(0, trainable=False)
    
    # create learning rate
    learning_rate = tf.train.exponential_decay(
                            0.01, 
                            global_step, 
                            100,     
                            0.95,    
                            staircase=True)
    
    # hidden layer 1
    hidden1 = tf.layers.dense(
        X,                              # input
        1024,                             # 64 units
        activation=tf.nn.relu,          # activation
        kernel_initializer=tf.variance_scaling_initializer(scale=2, seed=0),  # kernel initializer
        bias_initializer=tf.zeros_initializer(), # bias
        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
        name='hidden1'                   # name
    )
    
    hidden2 = tf.layers.dense(
        hidden1,                         # input
        56,                             # 48 units
        activation=tf.nn.relu,          # activation
        kernel_initializer=tf.variance_scaling_initializer(scale=2, seed=0),  # kernel initializer
        bias_initializer=tf.zeros_initializer(), # bias
        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
        name='hidden2'                  # name
    )
    
    logits = tf.layers.dense(
        hidden2,                       # input
        8,                             # 4 units
        activation=None,               # No activation function
        kernel_initializer=tf.variance_scaling_initializer(scale=1, seed=0),
        bias_initializer=tf.zeros_initializer(),
        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01),
        name='output'
    )
    
    # cross entropy and loss
    mean_ce = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))
    loss = mean_ce + tf.losses.get_regularization_loss()
    
    gd = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    # Minimize loss
    train_op = gd.minimize(loss, global_step=global_step)

    # Compute predictions and accuracy
    predictions = tf.argmax(logits, axis=1, output_type=tf.int32)
    is_correct = tf.equal(y, predictions)
    accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))
    
    # extra ops if we add batch norm
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    # Create summary hooks
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cross_entropy', mean_ce)
    tf.summary.scalar('learning_rate', learning_rate)
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()

In [53]:
## CONFIGURE OPTIONS
init = True                   # whether to initialize the model or use a saved version

meta_data_every = 5
log_to_tensorboard = True
print_every = 3                # how often to print metrics
checkpoint_every = 1           # how often to save model in epochs
use_gpu = False                 # whether or not to use the GPU
print_metrics = False          # whether to print or plot metrics, if False a plot will be created and updated every epoch
epochs = 1000
batch_size = 256

# Placeholders for metrics
if init:
    valid_acc_values = []
    valid_cost_values = []
    train_acc_values = []
    train_cost_values = []
    train_lr_values = []
    train_loss_values = []

if use_gpu:
    config = tf.ConfigProto()
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
else:
    config = tf.ConfigProto(device_count = {'GPU': 0})

In [ ]:
# train the model
with tf.Session(graph=graph, config=config) as sess:
    if log_to_tensorboard:
        train_writer = tf.summary.FileWriter('./logs/tr_' + model_name, sess.graph)
        test_writer = tf.summary.FileWriter('./logs/te_' + model_name)
    
    if not print_metrics:
        # create a plot to be updated as model is trained
        f, ax = plt.subplots(1,3,figsize=(20,5))
    
    # create the saver
    saver = tf.train.Saver()
    
    # If the model is new initialize variables, else restore the session
    if init:
        sess.run(tf.global_variables_initializer())
    else:
        saver.restore(sess, './model/cifar_'+model_name+'.ckpt')

    # Set seed
    np.random.seed(0)
    
    print("Training", model_name, "...")
    
    # Train several epochs
    for epoch in range(epochs):
        # Accuracy values (train) after each batch
        batch_acc = []
        batch_cost = []
        batch_loss = []
        batch_lr = []
        
        # only log run metadata once per epoch
        write_meta_data = False
            
        for X_batch, y_batch in get_batches(X_tr, y_tr, batch_size):
            if write_meta_data and log_to_tensboard:
                # create the metadata
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
            
                # Run training and evaluate accuracy
                _, _, summary, acc_value, cost_value, loss_value, step, lr = sess.run([train_op, extra_update_ops, merged, accuracy, mean_ce, loss, global_step, learning_rate], feed_dict={
                    X: X_batch,
                    y: y_batch,
                    training: True
                },
                options=run_options,
                run_metadata=run_metadata)

                # Save accuracy (current batch)
                batch_acc.append(acc_value)
                batch_cost.append(cost_value)
                batch_lr.append(lr)
                batch_loss.append(loss_value)
  
                # write the summary
                train_writer.add_run_metadata(run_metadata, 'step %d' % step)
                train_writer.add_summary(summary, step)
                write_meta_data = False
                
            else:
                # Run training without meta data
                _, _, summary, acc_value, cost_value, loss_value, step, lr = sess.run([train_op, extra_update_ops, merged, accuracy, mean_ce, loss, global_step, learning_rate], feed_dict={
                    X: X_batch,
                    y: y_batch,
                    training: True
                })

                # Save accuracy (current batch)
                batch_acc.append(acc_value)
                batch_cost.append(cost_value)
                batch_lr.append(lr)
                batch_loss.append(loss_value)
                
                # write the summary
                if log_to_tensorboard:
                    train_writer.add_summary(summary, step)

        # save checkpoint every nth epoch
        if(epoch % checkpoint_every == 0):
            print("Saving checkpoint")
            # save the model
            save_path = saver.save(sess, './model/cifar_'+model_name+'.ckpt')
    
            # Now that model is saved set init to false so we reload it
            init = False
        
        # init batch arrays
        batch_cv_acc = []
        batch_cv_cost = []
        batch_cv_loss = []
        
        # Evaluate validation accuracy with batches so as to not crash the GPU
        for X_batch, y_batch in get_batches(X_cv, y_cv, batch_size):
            summary, valid_acc, valid_cost, valid_loss = sess.run([merged, accuracy, mean_ce, loss], feed_dict={
                X: X_batch,
                y: y_batch,
                training: False
            })

            batch_cv_acc.append(valid_acc)
            batch_cv_cost.append(valid_cost)
            batch_cv_loss.append(valid_loss)

        # Write average of validation data to summary logs
        if log_to_tensorboard:
            summary = tf.Summary(value=[tf.Summary.Value(tag="accuracy", simple_value=np.mean(batch_cv_acc)),tf.Summary.Value(tag="cross_entropy", simple_value=np.mean(batch_cv_cost)),])
            test_writer.add_summary(summary, step)
            step += 1
            
        # take the mean of the values to add to the metrics
        valid_acc_values.append(np.mean(batch_cv_acc))
        valid_cost_values.append(np.mean(batch_cv_cost))
        train_acc_values.append(np.mean(batch_acc))
        train_cost_values.append(np.mean(batch_cost))
        train_lr_values.append(np.mean(batch_lr))
        train_loss_values.append(np.mean(batch_loss))
        
        if print_metrics:
            # Print progress every nth epoch to keep output to reasonable amount
            if(epoch % print_every == 0):
                print('Epoch {:02d} - step {} - cv acc: {:.3f} - train acc: {:.3f} (mean) - cv cost: {:.3f} - lr: {:.5f}'.format(
                    epoch, step, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost), lr
                ))
        else:
            # update the plot
            ax[0].cla()
            ax[0].plot(valid_acc_values, color="red", label="Validation")
            ax[0].plot(train_acc_values, color="blue", label="Training")
            ax[0].set_title('Validation accuracy: {:.4f} (mean last 3)'.format(np.mean(valid_acc_values[-3:])))
            
            # since we can't zoom in on plots like in tensorboard, scale y axis to give a decent amount of detail
            if np.mean(valid_acc_values[-3:]) > 0.85:
                ax[0].set_ylim([0.75,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.75:
                ax[0].set_ylim([0.65,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.65:
                ax[0].set_ylim([0.55,1.0])
            elif np.mean(valid_acc_values[-3:]) > 0.55:
                ax[0].set_ylim([0.45,1.0])           
            
            ax[0].set_xlabel('Epoch')
            ax[0].set_ylabel('Accuracy')
            ax[0].legend()
            
            ax[1].cla()
            ax[1].plot(valid_cost_values, color="red", label="Validation")
            ax[1].plot(train_cost_values, color="blue", label="Training")
            ax[1].set_title('Validation xentropy: {:.3f} (mean last 3)'.format(np.mean(valid_cost_values[-3:])))
            ax[1].set_xlabel('Epoch')
            ax[1].set_ylabel('Cross Entropy')
            ax[1].legend()
            
            ax[2].cla()
            ax[2].plot(train_lr_values)
            ax[2].set_title("Learning rate: {:.6f}".format(np.mean(train_lr_values[-1:])))
            ax[2].set_xlabel("Epoch")
            ax[2].set_ylabel("Learning Rate")
            
            display.display(plt.gcf())
            display.clear_output(wait=True)
            
        # Print data every 50th epoch so I can write it down to compare models
        if (not print_metrics) and (epoch % 50 == 0) and (epoch > 1):
            if(epoch % print_every == 0):
                print('Epoch {:02d} - step {} - cv acc: {:.3f} - train acc: {:.3f} (mean) - cv cost: {:.3f} - lr: {:.5f}'.format(
                    epoch, step, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost), lr
                ))  
            
    # print results of last epoch
    print('Epoch {} - cv acc: {:.4f} - train acc: {:.4f} (mean) - cv cost: {:.3f}'.format(
                epochs, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost)
            ))
    
    # save the session
    save_path = saver.save(sess, './model/cifar_'+model_name+'.ckpt')
    
    # init the test data array
    test_acc_values = []
    
    # Check on the test data
    for X_batch, y_batch in get_batches(X_te, y_te, batch_size):
        test_accuracy = sess.run(accuracy, feed_dict={
            X: X_batch,
            y: y_batch,
            training: False
        })
        test_acc_values.append(test_accuracy)
    
    # average test accuracy across batches
    test_acc = np.mean(test_acc_values)
    
# show the plot
plt.show()

# print results of last epoch
print('Epoch {} - cv acc: {:.4f} - train acc: {:.4f} (mean) - cv cost: {:.3f}'.format(
      epochs, np.mean(batch_cv_acc), np.mean(batch_acc), np.mean(batch_cv_cost)
    ))
    
# print test accuracy
#print("Convolutional network accuracy (test set):",test_acc, " Validation set:", valid_acc_values[-1])

Saving checkpoint


In [167]:
names

array(['blink', 'blinkInstruction', 'colorInstruction', 'colorRound',
       'math', 'mathInstruction', 'music', 'musicInstruction',
       'readyRound', 'relax', 'relaxInstruction', 'thinkOfItems',
       'thinkOfItemsInstruction', 'video', 'videoInstruction'],
      dtype=object)